In [169]:
import os
import sys
while not os.getcwd().endswith('ml'):
    os.chdir('..')
sys.path.insert(0, os.getcwd())

In [245]:
import pandas as pd
import numpy as np
import copy

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

#### Считывание данных

In [171]:
key = pd.read_csv("kaggle/walmart-recruiting-sales-in-stormy-weather/key.csv")
train = pd.read_csv("kaggle/walmart-recruiting-sales-in-stormy-weather/train.csv")
test = pd.read_csv("kaggle/walmart-recruiting-sales-in-stormy-weather/test.csv")
weather = pd.read_csv("kaggle/walmart-recruiting-sales-in-stormy-weather/weather.csv")

#### Описание данных

In [172]:
key.sample()

,store_nbr,station_nbr
35,36,18


In [173]:
train.sample()

,date,store_nbr,item_nbr,units
134291,2012-01-28,22,93,0


In [174]:
weather.sample()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
5654,7,2012-10-17,76,52,64,M,46,56,1,0,-,-,,0.0,0.00,28.29,29.61,6.3,29,15.7


In [175]:
weather.wetbulb.value_counts()

M      1252
74      777
75      684
73      663
72      609
71      526
70      481
67      474
65      459
64      437
62      433
68      428
66      425
63      423
60      419
61      409
59      408
58      406
69      399
56      383
76      362
55      356
57      344
52      337
54      330
53      325
44      323
45      307
50      305
51      304
       ... 
17       54
15       53
16       48
12       43
10       40
13       38
11       33
8        31
79       21
4        20
7        16
0        14
9        14
5        13
3        11
6         9
-3        7
2         7
1         5
-4        4
-2        3
-5        3
-7        2
-6        2
-10       2
-15       1
-8        1
-1        1
80        1
-9        1
Name: wetbulb, Length: 93, dtype: int64

#### Предобработка данных

In [223]:
def preprocessing(data, key, weather, target_field='units'):
    data = data.join(key.set_index("store_nbr"), on="store_nbr")
    data = data.join(weather.set_index(["station_nbr", "date"]), on=["station_nbr", "date"])
    data.drop(columns=["codesum", "sunrise", "sunset"])
    data.fillna(-1, inplace=True)
    
    needed_columns = ["store_nbr", "item_nbr"]
    if target_field in data.columns:
        needed_columns.append(target_field)
    
    data = data[needed_columns]
    data = pd.get_dummies(data, columns=["store_nbr", "item_nbr"])
    if target_field in data.columns:
        data['units'] = np.log(data['units'] + 1)
        
    return data

In [224]:
preprocessed_train = preprocessing(train, key, weather)
preprocessed_test = preprocessing(test, key, weather)

#### Обучение базовой модели

In [252]:
y = preprocessed_train['units']
X = preprocessed_train.loc[:, preprocessed_train.columns != 'units']

In [253]:
model = RandomForestRegressor(criterion='mse')

In [254]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

/Users/skrrydg/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [ ]:
model.fit(X_train, y_train)

/Users/skrrydg/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [ ]:
y_predict = model.predict(X_test)

In [ ]:
def get_score(y_true, y_predict):
    return mean_squared_error(y_true, y_predict)

In [ ]:
get_score(y_test, y_predict)

In [248]:
print(y_predict[np.abs(y_predict - y_test) > 0.1])
print(y_test[np.abs(y_predict - y_test) > 0.1])

[6.20457996]
1488312    6.359574
Name: units, dtype: float64


In [231]:
model.score(X_test, y_test)

0.9999999335966548